In [7]:
pip install transformers torch


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from transformers import pipeline, AutoTokenizer

# Models tried:
# CalvinHuang/mt5-small-finetuned-amazon-en-es (waaay too aggressive - glitchy if you ask it to produce larger summaries)
# Falconsai/text_summarization (not too bad, but will be aggressive if you don't give a large min size)
# google/pegasus-xsum (tries to do short summaries and then just repeats last sentence to pad to min length)
# facebook/bart-large-cnn (seems to do a decent job of balancing conciseness and coverage, not too aggressive)
#


text = "summarize: The Inflation Reduction Act lowers prescription drug costs, health care costs, and energy costs. It's the most aggressive action on tackling the climate crisis in American history, which will lift up American workers and create good-paying, union jobs across the country. It'll lower the deficit and ask the ultra-wealthy and corporations to pay their fair share. And no one making under $400,000 per year will pay a penny more in taxes."

summarizer = pipeline("summarization", model="dotslashderek/bart-large-cnn-prompt-summarization-v2")
summarizer(text)

/Users/dotslashderek/workspace/Gravitee/t5-summarization-testing/.venv/lib/python3.11/site-packages/transformers/models/bart/configuration_bart.py:177: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(
Device set to use mps:0
Your max_length is set to 142, but your input_length is only 94. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=47)


[{'summary_text': "The Inflation Reduction Act lowers prescription drug costs, health care costs, and energy costs. It's the most aggressive action on tackling the climate crisis in American history, which will lift up American workers and create good-paying, union jobs across the country. It'll lower the deficit and ask the ultra-wealthy and corporations to pay their fair share."}]

In [1]:
!pip install optimum onnxruntime
!pip install "optimum-onnx[onnxruntime]"

from optimum.pipelines import pipeline

summarizer = pipeline(
    "summarization",
    model="Xenova/bart-large-cnn",
    model_kwargs={"framework": "onnxruntime"}
)

text = "The Inflation Reduction Act lowers prescription drug costs, health care costs, and energy costs. It's the most aggressive action on tackling the climate crisis in American history, which will lift up American workers and create good-paying, union jobs across the country. It'll lower the deficit and ask the ultra-wealthy and corporations to pay their fair share. And no one making under $400,000 per year will pay a penny more in taxes."

summarizer(text)



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


/Users/dotslashderek/workspace/Gravitee/t5-summarization-testing/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/dotslashderek/workspace/Gravitee/t5-summarization-testing/.venv/lib/python3.11/site-packages/torch/onnx/_internal/registration.py:162: OnnxExporterWarning: Symbolic function 'aten::scaled_dot_product_attention' already registered for opset 14. Replacing the existing function with new function. This is unexpected. Please report it on https://github.com/pytorch/pytorch/issues.
  warnings.warn(


TypeError: optimum.onnxruntime.pipelines.ort_infer_framework_load_model() got multiple values for keyword argument 'framework'

In [27]:
import torch
from transformers import pipeline

summarizer = pipeline(
    task="summarization",
    model="google/pegasus-xsum",
    dtype=torch.float16,
    device=0
)

text = "The Inflation Reduction Act lowers prescription drug costs, health care costs, and energy costs. It's the most aggressive action on tackling the climate crisis in American history, which will lift up American workers and create good-paying, union jobs across the country. It'll lower the deficit and ask the ultra-wealthy and corporations to pay their fair share. And no one making under $400,000 per year will pay a penny more in taxes."

summarizer(text)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use mps:0


[{'summary_text': 'President Obama has signed into law one of the biggest tax cuts in American history.'}]

In [3]:
prompt_tuples = [(
    "summarize: I'm handing you a crusty legacy service that was written in Java 8 circa 2016 and sprinkled with singletons, static utils, and a homegrown futures thing. I need a straight path to modernizing it without breaking prod. Please sketch a refactor plan that: (1) migrates to Java 21 or Kotlin (I'm leaning Kotlin but convince me either way), (2) replaces our bespoke async with Project Loom or Kotlin coroutines, (3) pulls DI into something standard (we currently have Spring 4.x; is it worth jumping to Spring Boot 3.x or going Micronaut/Quarkus?), (4) adds modular tests (unit+contract) around our three hottest endpoints before touching anything, (5) introduces structured logging and OpenTelemetry tracing, (6) ships minimal slices behind flags so we can canary. Constraints: zero downtime, we’re on AWS (ECS + RDS Postgres), and we can only spare 20% engineering time for four sprints. Deliverables I want: phased plan, risks, a rollback story, and sample code for one endpoint rewritten with new concurrency + tracing.",
    "summary: Propose a phased modernization of a legacy Java 8 service—choose Java 21 vs Kotlin, replace custom async with Loom/coroutines, upgrade DI framework, add tests, observability, and feature-flagged canaries—within AWS constraints and limited team capacity."
),
(
    "summarize: I'm taking my partner to Japan for 10 days in late April (cherry blossoms likely tail-end). We're flying into Tokyo and out of Osaka. We love food markets, quiet temples, and one quirky museum per city. We hate rushing. Budget is mid, not luxury. Please build a day-by-day plan that: groups attractions by neighborhood to minimize transit, mixes high-energy days with slow mornings, includes 2 omakase options (one splurge, one affordable), flags any spots that need reservations, and gives exact JR/metro lines where it helps. We want one overnight in Hakone (onsen with private bath), a tea ceremony that isn’t tourist-trappy, and a half-day thrift hunt. Accessibility note: mild knee issues, so limit long stair climbs. Also suggest a compact packing list tuned for variable spring weather and guidance on eSIM vs pocket Wi-Fi. If rain hits, give alternates. Save us from FOMO—curate!",
    "summary: Create a relaxed 10-day Tokyo→Hakone→Kyoto/Osaka itinerary with curated food, temples, logistics (transit, reservations), accessibility considerations, packing tips, and rain-day alternates."
),
(
    "summarize: I need a three-email B2B nurture sequence for a mid-market SaaS (API observability). Persona: staff+ principal platform engineers who hate fluff, love concrete ROI, and are wary of lock-in. Tone: credible, technical, and crisp—no hype. Goals: (E1) problem framing with a 90-second demo link, (E2) technical deep-dive with a minimal POC guide (docker-compose + sample traffic), (E3) social proof + ‘path to value in 7 days’ checklist. Must handle objections (cost, data egress, vendor risk), and highlight open standards (OpenTelemetry), zero long-term contracts, and usage caps. Add strong but respectful CTAs, subject line variants, and preview text. Keep each email scannable (bullets allowed), with one visual suggestion per email. Compliance: CAN-SPAM basics and an obvious unsubscribe.",
    "summary: Draft a 3-email technical nurture for API observability—demo, POC guide, and social proof—addressing lock-in/cost objections, emphasizing OpenTelemetry and flexible pricing, with crisp CTAs and compliance."
),
(
    "summarize: I’ve got a messy CSV export (hundreds of MB) from a subscription app. Columns are inconsistent across months; some fields are camelCase, some snake_case; dates alternate between ISO and US formats; customerId sometimes missing but recoverable via email. I need a reproducible data-cleaning pipeline that: (a) normalizes schema, (b) dedupes on a stable user key, (c) parses plans/add-ons into a tidy table, (d) computes MRR, expansion, contraction, churn (logo + revenue) by month, and (e) outputs a dashboard-friendly parquet. Prefer Python with Pandas/Polars; happy with dbt if compelling. Please include file layout, dependency pinning, a Makefile, and a quick validation harness with sample assertions. Also note data privacy steps (PII hashing) and performance tips for memory-limited machines.",
    "summary: Specify a Python-based ETL to clean inconsistent subscription CSVs, normalize schema, dedupe users, compute SaaS metrics, export parquet, and include reproducibility, privacy, and performance details."
),
(
    "summarize: I want product requirements for a mobile feature called “Trips Auto-Organizer.” Inputs: emailed receipts, calendar events, and map location history. Output: a private trip timeline with expenses, docs, and offline access. Must: on-device extraction when possible, graceful degradation without network, and privacy-first defaults with explicit sharing. Edge cases: overlapping trips, multi-currency expenses with mid-market FX, and partial-day hops. Success metrics: setup < 2 min, <3 taps to find any doc, complaint rate <1%. Please write a PRD with problem, goals/non-goals, user stories, happy/sad paths, data model sketch, analytics, rollout plan, and a v1/v2 cut. Include accessibility and low-battery modes.",
    "summary: Draft a privacy-first PRD for “Trips Auto-Organizer”—ingesting receipts/calendars/locations to build offline trip timelines with robust edge cases, UX flows, data model, metrics, and phased rollout."
),
(
    "summarize: Conduct a lightweight security review of our AWS footprint. We’re a small team using two accounts (prod/stage), ECS Fargate, RDS Postgres, S3, and CloudFront. I want an actionable checklist that a mid-level dev can run in a day: IAM basics (least privilege, MFA, access keys rotation), network posture (SGs, NACLs), data at rest/in transit, backup/restore drills, logging/alerts (CloudTrail, GuardDuty), and secrets handling (SSM/Secrets Manager). Provide sample CLI commands and Terraform snippets to detect & fix common misconfigs. Include a ‘top 10 quick wins’ and a ‘next 30 days’ plan. Avoid vendor lock-in tooling; native AWS preferred.",
    "summary: Provide an AWS security hardening checklist with commands/snippets—cover IAM, network, encryption, backups, logging/alerts, and secrets—plus quick wins and a 30-day plan."
),
(
    "summarize: I need a 75-minute workshop plan to teach junior devs how to write good bug reports and reproduction cases. Context: distributed system with intermittent failures. Students struggle to separate symptoms from causes and rarely attach logs. Please include: learning objectives, a simple taxonomy (expected vs actual, scope, environment), a template they can paste in Jira, two practice exercises (one front-end, one backend), and a rubric for grading clarity. Provide a short section on capturing minimal logs/metrics (e.g., curl -v, browser HAR, correlation IDs), and teach how to create a minimal repro. End with a cheat sheet they can print.",
    "summary: Design a 75-min workshop for juniors on high-quality bug reports—taxonomy, Jira template, exercises, logging tips, repro techniques, and a printable cheat sheet."
),
(
    "summarize: I’m renovating a small kitchen (3.0m x 2.4m). Constraints: building is old, only one load-bearing wall; budget £9–12k all-in; we cook daily; want induction, decent ventilation, and durable surfaces. Please suggest a layout (galley vs L), storage strategy for small appliances, lighting plan (task + ambient), and a phased schedule to minimize downtime (temporary cooking setup). Provide a line-item budget (units, worktop, appliances, trades, permits, contingency). Include material choices with pros/cons (laminate vs quartz, flat-pack vs bespoke), and a checklist for snagging day. Also include energy-efficiency notes and where to splurge vs save.",
    "summary: Propose a practical small-kitchen renovation plan—layout, storage, lighting, phased schedule, itemized budget, material tradeoffs, efficiency tips, and splurge/save guidance."
),
(
    "summarize: Please critique and tighten our customer support policy. Current issues: inconsistent SLAs, scattered macros, and tone swings between robotic and chummy. Audience: global B2B users across time zones. Deliverables: a tiered SLA matrix (by severity and plan), an escalation ladder, 8–10 canonical macros rewritten with empathetic but concise language, and a style guide (greeting, apology, ownership, next step, close). Add guidance for ‘unknowns’ (acknowledge, time-box investigation), bad-news delivery, and when to hop on a call. Include QA metrics (FCR, CES, CSAT), cadences for reviewing tickets, and a changelog process.",
    "summary: Rewrite support policy with clear SLAs, escalation paths, consistent empathetic macros, tone/style guide, handling unknowns and bad news, plus QA metrics and review cadence."
),
(
    "summarize: I’m evaluating vector databases for a hybrid RAG system. Use case: millions of short code/doc chunks with frequent upserts and metadata filtering. Requirements: HNSW or IVF support, pq/opq options, strong filtering, hybrid BM25+vector, horizontal scale, and decent Python/Java clients. Candidates: Elastic, OpenSearch, Qdrant, Milvus, Weaviate, pgvector. Please produce a comparison table (features, recall/latency trade-offs, maturity, ops complexity, cloud vs self-host), a minimal benchmark plan (dataset, metrics, queries), and a decision rubric weighted for our needs (filtering, write throughput, TCO). End with a safe default and migration risks.",
    "summary: Compare major vector stores for hybrid RAG—features, performance, ops, ecosystem—propose a small benchmark plan, a weighted decision rubric, and recommend a safe default with migration caveats."
)]


In [5]:
for i, (prompt, reference_summary) in enumerate(prompt_tuples):
    print(f"\nExample {i+1}:")
    print("Prompt:", prompt)
    result = summarizer(prompt, max_length=200, min_length=10, do_sample=False)
    print("Generated Summary:", result[0]['summary_text'])
    print("Reference Summary:", reference_summary)



Both `max_new_tokens` (=256) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



Example 1:
Prompt: summarize: I'm handing you a crusty legacy service that was written in Java 8 circa 2016 and sprinkled with singletons, static utils, and a homegrown futures thing. I need a straight path to modernizing it without breaking prod. Please sketch a refactor plan that: (1) migrates to Java 21 or Kotlin (I'm leaning Kotlin but convince me either way), (2) replaces our bespoke async with Project Loom or Kotlin coroutines, (3) pulls DI into something standard (we currently have Spring 4.x; is it worth jumping to Spring Boot 3.x or going Micronaut/Quarkus?), (4) adds modular tests (unit+contract) around our three hottest endpoints before touching anything, (5) introduces structured logging and OpenTelemetry tracing, (6) ships minimal slices behind flags so we can canary. Constraints: zero downtime, we’re on AWS (ECS + RDS Postgres), and we can only spare 20% engineering time for four sprints. Deliverables I want: phased plan, risks, a rollback story, and sample code for one 

Both `max_new_tokens` (=256) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Generated Summary: I need a straight path to modernizing it without breaking prod
Reference Summary: summary: Propose a phased modernization of a legacy Java 8 service—choose Java 21 vs Kotlin, replace custom async with Loom/coroutines, upgrade DI framework, add tests, observability, and feature-flagged canaries—within AWS constraints and limited team capacity.

Example 2:
Prompt: summarize: I'm taking my partner to Japan for 10 days in late April (cherry blossoms likely tail-end). We're flying into Tokyo and out of Osaka. We love food markets, quiet temples, and one quirky museum per city. We hate rushing. Budget is mid, not luxury. Please build a day-by-day plan that: groups attractions by neighborhood to minimize transit, mixes high-energy days with slow mornings, includes 2 omakase options (one splurge, one affordable), flags any spots that need reservations, and gives exact JR/metro lines where it helps. We want one overnight in Hakone (onsen with private bath), a tea ceremony tha

Both `max_new_tokens` (=256) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

Generated Summary: I need a 75-minute workshop plan to teach junior devs how to write good reports and reproduction cases
Reference Summary: summary: Design a 75-min workshop for juniors on high-quality bug reports—taxonomy, Jira template, exercises, logging tips, repro techniques, and a printable cheat sheet.

Example 8:
Prompt: summarize: I’m renovating a small kitchen (3.0m x 2.4m). Constraints: building is old, only one load-bearing wall; budget £9–12k all-in; we cook daily; want induction, decent ventilation, and durable surfaces. Please suggest a layout (galley vs L), storage strategy for small appliances, lighting plan (task + ambient), and a phased schedule to minimize downtime (temporary cooking setup). Provide a line-item budget (units, worktop, appliances, trades, permits, contingency). Include material choices with pros/cons (laminate vs quartz, flat-pack vs bespoke), and a checklist for snagging day. Also include energy-efficiency notes and where to splurge vs save.


Your max_length is set to 200, but your input_length is only 184. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=92)
Both `max_new_tokens` (=256) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Generated Summary: I’m renovating a small kitchen (3.0m x 2.4m). Constraints: building is old, only one load-bearing wall ...
Reference Summary: summary: Propose a practical small-kitchen renovation plan—layout, storage, lighting, phased schedule, itemized budget, material tradeoffs, efficiency tips, and splurge/save guidance.

Example 9:
Prompt: summarize: Please critique and tighten our customer support policy. Current issues: inconsistent SLAs, scattered macros, and tone swings between robotic and chummy. Audience: global B2B users across time zones. Deliverables: a tiered SLA matrix (by severity and plan), an escalation ladder, 8–10 canonical macros rewritten with empathetic but concise language, and a style guide (greeting, apology, ownership, next step, close). Add guidance for ‘unknowns’ (acknowledge, time-box investigation), bad-news delivery, and when to hop on a call. Include QA metrics (FCR, CES, CSAT), cadences for reviewing tickets, and a changelog process.


Your max_length is set to 200, but your input_length is only 193. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=96)
Both `max_new_tokens` (=256) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Generated Summary: Please critique and tighten our customer support policy
Reference Summary: summary: Rewrite support policy with clear SLAs, escalation paths, consistent empathetic macros, tone/style guide, handling unknowns and bad news, plus QA metrics and review cadence.

Example 10:
Prompt: summarize: I’m evaluating vector databases for a hybrid RAG system. Use case: millions of short code/doc chunks with frequent upserts and metadata filtering. Requirements: HNSW or IVF support, pq/opq options, strong filtering, hybrid BM25+vector, horizontal scale, and decent Python/Java clients. Candidates: Elastic, OpenSearch, Qdrant, Milvus, Weaviate, pgvector. Please produce a comparison table (features, recall/latency trade-offs, maturity, ops complexity, cloud vs self-host), a minimal benchmark plan (dataset, metrics, queries), and a decision rubric weighted for our needs (filtering, write throughput, TCO). End with a safe default and migration risks.
Generated Summary: I’m evaluating vec

In [12]:
# Install evaluate if not already installed
try:
    import evaluate
except ImportError:
    import sys
    !{sys.executable} -m pip install evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached evaluate-0.4.6-py3-none-any.whl.metadata (9.5 kB)
  Using cached dill-0.4.0-py3-none-any.whl.metadata (10 kB)
  Using cached pyarrow-21.0.0-cp311-cp311-macosx_12_0_arm64.whl.metadata (3.3 kB)
  Using cached multiprocess-0.70.16-py311-none-any.whl.metadata (7.2 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.4.0-py3-none-any.whl.metadata (3.7 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached evaluate-0.4.6-py3-none-any.whl (84 kB)
Using cached dill-0.4.0-py3-none-any.whl (119 kB)
Using cached multiprocess-0.70.16-py311-none-any.whl (143 kB)
Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl (15 kB)
Using cached aiosignal-1.4.0-py3-none-any.whl (7.5 kB)
Using cached pyarrow-21.0.0-cp311-cp311-macosx_12_0_arm64.whl (31.2 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 84.3 MB/s eta 0:00:00
Using cached pyt

In [4]:
from evaluate import load
from transformers import pipeline, AutoTokenizer
rouge = load('rouge')

tokenizer = AutoTokenizer.from_pretrained("dotslashderek/my_awesome_billsum_model")

generated_summaries = []
reference_summaries = []

for i, (prompt, reference_summary) in enumerate(prompt_tuples):
    # Tokenize prompt to get input length in tokens
    input_tokens = tokenizer(prompt, return_tensors='pt')['input_ids'][0]
    input_length = len(input_tokens)
    # Calculate dynamic max/min lengths
    max_length = max(1, int(input_length * 0.90))
    min_length = max(1, int(input_length * 0.50))
    if min_length >= max_length:
        min_length = max(1, max_length - 1)
    result = summarizer(prompt, max_length=max_length, min_length=min_length, do_sample=False, early_stopping=False)
    summary_tokens = tokenizer(result[0]['summary_text'], return_tensors='pt')['input_ids'][0]
    generated = result[0]['summary_text']
    generated_summaries.append(generated)
    reference_summaries.append(reference_summary)
    print(f"\nExample {i+1}:")
    print("Prompt:", prompt)
    print(f"Tokenized input length: {input_length}, max_length: {max_length}, min_length: {min_length}")
    print("Generated Summary:", generated)
    print(f"Generated token length: {len(summary_tokens)}")
    print("Reference Summary:", reference_summary)

# Compute ROUGE scores for all examples
scores = rouge.compute(predictions=generated_summaries, references=reference_summaries)
print("\nROUGE scores (aggregated):")
for k, v in scores.items():
    print(f"{k}: {v:.4f}")

# Optionally, print ROUGE for each example
print("\nROUGE-L for each example:")
for i, (gen, ref) in enumerate(zip(generated_summaries, reference_summaries)):
    score = rouge.compute(predictions=[gen], references=[ref])
    print(f"Example {i+1}: ROUGE-L: {score['rougeL']:.4f}")


Your max_length is set to 252, but your input_length is only 250. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=125)



Example 1:
Prompt: summarize: I'm handing you a crusty legacy service that was written in Java 8 circa 2016 and sprinkled with singletons, static utils, and a homegrown futures thing. I need a straight path to modernizing it without breaking prod. Please sketch a refactor plan that: (1) migrates to Java 21 or Kotlin (I'm leaning Kotlin but convince me either way), (2) replaces our bespoke async with Project Loom or Kotlin coroutines, (3) pulls DI into something standard (we currently have Spring 4.x; is it worth jumping to Spring Boot 3.x or going Micronaut/Quarkus?), (4) adds modular tests (unit+contract) around our three hottest endpoints before touching anything, (5) introduces structured logging and OpenTelemetry tracing, (6) ships minimal slices behind flags so we can canary. Constraints: zero downtime, we’re on AWS (ECS + RDS Postgres), and we can only spare 20% engineering time for four sprints. Deliverables I want: phased plan, risks, a rollback story, and sample code for one 

Your max_length is set to 207, but your input_length is only 198. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=99)



Example 3:
Prompt: summarize: I need a three-email B2B nurture sequence for a mid-market SaaS (API observability). Persona: staff+ principal platform engineers who hate fluff, love concrete ROI, and are wary of lock-in. Tone: credible, technical, and crisp—no hype. Goals: (E1) problem framing with a 90-second demo link, (E2) technical deep-dive with a minimal POC guide (docker-compose + sample traffic), (E3) social proof + ‘path to value in 7 days’ checklist. Must handle objections (cost, data egress, vendor risk), and highlight open standards (OpenTelemetry), zero long-term contracts, and usage caps. Add strong but respectful CTAs, subject line variants, and preview text. Keep each email scannable (bullets allowed), with one visual suggestion per email. Compliance: CAN-SPAM basics and an obvious unsubscribe.
Tokenized input length: 219, max_length: 197, min_length: 109
Generated Summary: I need a three-email B2B nurture sequence for a mid-market SaaS (API observability). Persona: sta

Your max_length is set to 174, but your input_length is only 170. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=85)


...


Example 6:
Prompt: summarize: Conduct a lightweight security review of our AWS footprint. We’re a small team using two accounts (prod/stage), ECS Fargate, RDS Postgres, S3, and CloudFront. I want an actionable checklist that a mid-level dev can run in a day: IAM basics (least privilege, MFA, access keys rotation), network posture (SGs, NACLs), data at rest/in transit, backup/restore drills, logging/alerts (CloudTrail, GuardDuty), and secrets handling (SSM/Secrets Manager). Provide sample CLI commands and Terraform snippets to detect & fix common misconfigs. Include a ‘top 10 quick wins’ and a ‘next 30 days’ plan. Avoid vendor lock-in tooling; native AWS preferred.
Tokenized input length: 193, max_length: 173, min_length: 96
Generated Summary: Create a lightweight AWS security review checklist for a small team: IAM basics (least privilege, MFA, access keys rotation), network posture (SGs, NACLs), data at rest/in transit, backup/restore drills, logging/alerts (CloudTrail, GuardDuty), se

Your max_length is set to 170, but your input_length is only 162. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=81)



Example 7:
Prompt: summarize: I need a 75-minute workshop plan to teach junior devs how to write good bug reports and reproduction cases. Context: distributed system with intermittent failures. Students struggle to separate symptoms from causes and rarely attach logs. Please include: learning objectives, a simple taxonomy (expected vs actual, scope, environment), a template they can paste in Jira, two practice exercises (one front-end, one backend), and a rubric for grading clarity. Provide a short section on capturing minimal logs/metrics (e.g., curl -v, browser HAR, correlation IDs), and teach how to create a minimal repro. End with a cheat sheet they can print.
Tokenized input length: 163, max_length: 146, min_length: 81
Generated Summary: Create a 75-minute workshop plan for junior devs: learning objectives, taxonomy (expected vs actual, scope, environment), Jira template, two practice exercises (front-end, backend), grading rubric, capturing minimal logs/metrics ( curl -v, browse

Your max_length is set to 171, but your input_length is only 164. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=82)



Example 9:
Prompt: summarize: Please critique and tighten our customer support policy. Current issues: inconsistent SLAs, scattered macros, and tone swings between robotic and chummy. Audience: global B2B users across time zones. Deliverables: a tiered SLA matrix (by severity and plan), an escalation ladder, 8–10 canonical macros rewritten with empathetic but concise language, and a style guide (greeting, apology, ownership, next step, close). Add guidance for ‘unknowns’ (acknowledge, time-box investigation), bad-news delivery, and when to hop on a call. Include QA metrics (FCR, CES, CSAT), cadences for reviewing tickets, and a changelog process.
Tokenized input length: 171, max_length: 153, min_length: 85
Generated Summary: Deliver a curated B2B support policy review plan with tiered SLA matrix, escalation ladder, 8–10 macros rewritten with empathetic language, style guide (greeting, apology, ownership, next step, close), guidance for ‘unknowns’ (acknowledge, time-box investigation),